In [ ]:
import numpy as np 
import pytorch_lightning as pl 
import torch 
from torch.utils.data import DataLoader , Dataset 

In [ ]:
# importing required stuff
# !pip -q install --upgrade wandb
import pandas as pd
import albumentations
import torchvision 
from PIL import Image
import wandb
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import torch 
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
wandb.login(key = user_secrets.get_secret("wandb_key"))
import cv2
from sklearn.model_selection import train_test_split
import pytorch_lightning as pl 
import torch 
import numpy as np 
import pandas as pd
import  os 
import sys 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler
sys.path.append("../input/timm-pytorch-image-models/pytorch-image-models-master")
import timm
import math
import matplotlib.pyplot as plt
import torchvision

# run = wandb.init(project = "hyperwhaletpu",name = "pytorchlightning")

train = pd.read_csv("../input/happy-whale-and-dolphin/train.csv")
ss = pd.read_csv("../input/happy-whale-and-dolphin/sample_submission.csv")
train['image'] = train["image"].apply(lambda x: f"../input/happy-whale-and-dolphin/train_images/{x}")
ss['image'] = ss['image'].apply(lambda x : f"../input/happy-whale-and-dolphin/test_images/{x}")
# train = train.sample(frac = 0.001)
landmark_id2idx = {landmark_id: idx for idx, landmark_id in enumerate(sorted(train['individual_id'].unique()))}
idx2landmark_id = {idx: landmark_id for idx, landmark_id in enumerate(sorted(train['individual_id'].unique()))}
train['individual_id'] = train['individual_id'].map(landmark_id2idx)
# val['individual_id'] = val['individual_id'].map(landmark_id2idx)

train,val = train_test_split(train)





transforms_224 = albumentations.Compose([
    albumentations.Resize(224,224),
    albumentations.Normalize()
])

transforms_768 = albumentations.Compose([
    albumentations.Resize(768, 768),
    albumentations.Normalize()
    
    
    
    
    
    
])
transforms_512 = albumentations.Compose([
    albumentations.Resize(512, 512),
    albumentations.Normalize()
])


# creating the pytroch model AlexModel 
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, file_paths , labels , transforms):
        # Not sure if I am gonnaa use species but let's see 
        self.file_paths = file_paths 
        self.labels = labels 
        self.transforms = transforms
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self,idx):
        lab = self.labels[idx]
        image = cv2.imread(self.file_paths[idx])
        image = image[:, :, ::-1]
        res = self.transforms(image=image)
        image = res['image'].astype(np.float32)
        image = image.transpose(2, 0, 1)      
        return {"image":image, "label":lab}
    
class test_dataset(torch.utils.data.Dataset):
    def __init__(self, file_paths , transforms):
        # Not sure if I am gonnaa use species but let's see 
        self.file_paths = file_paths 
        self.transforms = transforms
    
    def __len__(self):
        return len(self.file_paths)
    
    def __getitem__(self,idx):
        image = cv2.imread(self.file_paths[idx])
        image = image[:, :, ::-1]
        res = self.transforms(image=image)
        image = res['image'].astype(np.float32)
        image = image.transpose(2, 0, 1)      
        return {"image":image}

class ArcMarginProduct_subcenter(nn.Module):
    def __init__(self, in_features, out_features, k=3):
        super().__init__()
        self.weight = nn.Parameter(torch.FloatTensor(out_features*k, in_features))
        self.reset_parameters()
        self.k = k
        self.out_features = out_features
        
    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        
    def forward(self, features):
        cosine_all = F.linear(F.normalize(features), F.normalize(self.weight))
        cosine_all = cosine_all.view(-1, self.out_features, self.k)
        cosine, _ = torch.max(cosine_all, dim=2)
        return cosine   

class DenseCrossEntropy(nn.Module):
    def forward(self, x, target):
        x = x.float()
        target = target.float()
        logprobs = torch.nn.functional.log_softmax(x, dim=-1)

        loss = -logprobs * target
        loss = loss.sum(-1)
        return loss.mean()
class ArcFaceLossAdaptiveMargin(nn.modules.Module):
    def __init__(self, margins, s=30.0):
        super().__init__()
        self.crit =DenseCrossEntropy()
        self.s = s
        self.margins = margins
            
    def forward(self, logits, labels, out_dim=15587):
#         device = "cuda" if torch.cuda.is_available() else "cpu"
#         device = xm.xla_device()
        ms = []
        
        ms = self.margins[labels.cpu().numpy()]
        cos_m = torch.from_numpy(np.cos(ms)).float()
        sin_m = torch.from_numpy(np.sin(ms)).float()
        th = torch.from_numpy(np.cos(math.pi - ms)).float()
        mm = torch.from_numpy(np.sin(math.pi - ms) * ms).float()
        labels = F.one_hot(labels, out_dim).float()
        logits = logits.float()
        cosine = logits
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        phi = cosine * cos_m.view(-1,1) - sine * sin_m.view(-1,1)
        phi = torch.where(cosine > th.view(-1,1), phi, cosine - mm.view(-1,1))
        output = (labels * phi) + ((1.0 - labels) * cosine)
        output *= self.s
        loss = self.crit(output, labels)
        return loss  

tmp = np.sqrt(1 / np.sqrt(train.individual_id.value_counts().sort_index().values))
margins = (tmp - tmp.min()) / (tmp.max() - tmp.min()) * 0.45 + 0.05

transforms_224 = albumentations.Compose([
    albumentations.Resize(224,224),
    albumentations.Normalize()
])

# train_ds = MyDataset(file_paths = train.image, labels = train.individual_id, transforms = transforms_224)
# train_dl = DataLoader(train_ds, batch_size=10, num_workers = 4 , pin_memory = True, drop_last = True)

# val_ds = MyDataset(file_paths = val.image, labels = val.individual_id, transforms = transforms_224)
# val_dl = DataLoader(val_ds,batch_size = 10, num_workers = 4 , pin_memory = True, drop_last = False)

class AlexModel(pl.LightningModule):
    def __init__(self, batch_size):
        super(AlexModel,self).__init__()
        self.backbone = timm.create_model("efficientnet_b5",pretrained = True) 
        self.linear = nn.Linear(1000, 512)
        self.relu = nn.LeakyReLU()
        self.out_layer = ArcMarginProduct_subcenter(512, 15587)
        self.softmax = nn.Softmax(dim = 0)
        self.save_hyperparameters()
        self.batch_size = batch_size
        
    
    def forward(self, input_image):
        x = self.backbone(input_image)
        x = self.relu(self.linear(x))
        out_logits = self.softmax(self.out_layer(x) )
        
        return out_logits
    
    def training_step(self,batch,batch_idx):
        data = batch["image"]
        target = batch["label"]

        logits_m = self(data)
#         print(logits_m.shape, target.shape)
        
        loss = nn.CrossEntropyLoss()(logits_m,torch.tensor(target, dtype = torch.long))
#         arc = ArcFaceLossAdaptiveMargin(margins=margins, s=80)
#         loss = arc(logits_m, target, 15587)
        return {"loss":loss}
    
    def validation_step(self,batch,batch_idx):
        data = batch["image"]
        target = batch["label"]
        logits_m = self(data)
#         print(logits_m.shape, target.shape)
        loss = nn.CrossEntropyLoss()(logits_m, torch.tensor(target,dtype = torch.long))
        self.log('val_loss', loss)
        return {"val_loss":loss, "pred":logits_m}
    
    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
#         tensorboard_logs = {'val_loss': avg_loss}
        return {'avg_val_loss': avg_loss}
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.02)

    def train_dataloader(self):
        dataset =  MyDataset(file_paths = train.image.to_numpy(), labels = train.individual_id.to_numpy(), transforms = transforms_224)
        loader = DataLoader(dataset, batch_size=self.batch_size)
        return loader

    def val_dataloader(self):
        dataset = MyDataset(file_paths = val.image.to_numpy(), labels = val.individual_id.to_numpy(), transforms = transforms_224)
        loader = DataLoader(dataset, batch_size=self.batch_size)
        return loader
    
    def predict_dataloader(self):
        dataset = test_dataset(file_paths = ss.image.to_numpy(), transforms = transforms_224)
        loader = DataLoader(dataset , batch_size = self.batch_size)
        return loader 

    def predict_step(self, batch , batch_idx):
        data = batch["image"]
        logits_m = self(data)
        return logits_m


In [ ]:
mod = AlexModel(10)

In [ ]:
mod.load_state_dict(torch.load("../input/happywhale-pytorch-lightning-model-building/AlexModelHap.pt"))

In [ ]:
from pytorch_lightning.loggers import WandbLogger  # newline 1
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
es = EarlyStopping(monitor="val_loss")
wandb_logger = WandbLogger() 
checkpointing = ModelCheckpoint(monitor="val_loss")
trainer = pl.Trainer(auto_scale_batch_size=True,logger=wandb_logger,callbacks=[es,checkpointing],progress_bar_refresh_rate=20,gpus = 0 , max_epochs = 7)
# trainer.fit(mod)

In [ ]:
# torch.save(mod.state_dict(),"./AlexModelHap.pt")

In [ ]:
preds = trainer.predict(mod)

In [ ]:
preds

In [ ]:
landmarks = []
for pred in preds:
    pred_sorted_last_5 = pred.argsort(axis=1)[:,-4:]
    for val in pred_sorted_last_5:
        try:
            ans = [idx2landmark_id[x.item()] for x in val]
        except :
            ans = val.numpy()
#             ans = list(ans.numpy())
        ans.append("new_individual")
        ans = ' '.join(ans)
        landmarks.append(ans)

# preds[0].argsort(axis = 1)[:,-5:]

In [ ]:
len(landmarks)

In [ ]:
landmarks

In [ ]:
ss = pd.read_csv('../input/happy-whale-and-dolphin/sample_submission.csv')
ss

In [ ]:
ss['predictions'] = landmarks

In [ ]:
ss

In [ ]:
ss.to_csv('submission.csv', index = False)